# 多项式回归

本笔记本实现了多项式回归模型，展示了如何处理非线性数据，以及不同阶数多项式的拟合效果和过拟合问题。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. 生成非线性数据

In [ ]:
def generate_nonlinear_data():
    """
    生成非线性数据，用于多项式回归
    
    返回:
    X: 特征数据 (n_samples, 1)
    y: 目标值 (n_samples, 1)，包含立方关系和噪声
    """
    n_samples = 100
    # 生成-3到3之间的随机特征值
    X = 6 * np.random.rand(n_samples, 1) - 3
    # 生成非线性目标值（立方关系），添加高斯噪声
    y = 0.5 * X**3 + X**2 + X + 2 + np.random.randn(n_samples, 1) * 2
    return X, y

# 生成数据
X, y = generate_nonlinear_data()
print(f"数据形状: X={X.shape}, y={y.shape}")

## 2. 可视化原始数据

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6)
plt.xlabel('特征 X')
plt.ylabel('目标值 y')
plt.title('原始非线性数据')
plt.grid(True)
plt.show()

## 3. 创建多项式特征

In [ ]:
def create_polynomial_features(X, degree):
    """
    创建多项式特征
    
    参数:
    X: 原始特征矩阵 (m, 1)
    degree: 多项式次数
    
    返回:
    X_poly: 多项式特征矩阵 (m, degree+1)
    """
    # 初始化多项式特征矩阵，包含偏置项
    X_poly = np.ones((len(X), 1))
    # 添加各次幂的特征
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, np.power(X, d)))
    return X_poly

## 4. 线性回归实现（用于多项式回归）

In [ ]:
def linear_regression(X, y):
    """
    使用正规方程求解线性回归
    
    参数:
    X: 特征矩阵 (m, n+1)
    y: 目标值向量 (m, 1)
    
    返回:
    theta: 学习到的参数向量 (n+1, 1)
    """
    # 使用正规方程求解参数
    theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    return theta

## 5. 模型预测和评估

In [ ]:
def predict(X, theta):
    """
    使用学习到的参数进行预测
    
    参数:
    X: 特征矩阵 (m, n+1)
    theta: 参数向量 (n+1, 1)
    
    返回:
    predictions: 预测值向量 (m, 1)
    """
    return X.dot(theta)

def compute_mse(y_true, y_pred):
    """
    计算均方误差
    
    参数:
    y_true: 真实目标值 (m, 1)
    y_pred: 预测值 (m, 1)
    
    返回:
    mse: 均方误差值
    """
    return np.mean(np.square(y_true - y_pred))

## 6. 不同阶数多项式回归的拟合效果

In [ ]:
degrees = [1, 2, 3, 10]
plt.figure(figsize=(14, 10))

for i, degree in enumerate(degrees):
    # 创建多项式特征
    X_poly = create_polynomial_features(X, degree)
    # 训练模型
    theta = linear_regression(X_poly, y)
    
    # 生成测试数据用于绘制曲线
    X_new = np.linspace(-3, 3, 100).reshape(100, 1)
    X_new_poly = create_polynomial_features(X_new, degree)
    y_pred = predict(X_new_poly, theta)
    
    # 计算训练集上的MSE
    y_train_pred = predict(X_poly, theta)
    mse = compute_mse(y, y_train_pred)
    
    # 绘制结果
    plt.subplot(2, 2, i + 1)
    plt.scatter(X, y, alpha=0.6, label='训练数据')
    plt.plot(X_new, y_pred, 'r-', linewidth=2, label=f'{degree}阶多项式')
    plt.xlabel('特征 X')
    plt.ylabel('目标值 y')
    plt.title(f'{degree}阶多项式回归 (MSE: {mse:.2f})')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

## 7. 过拟合分析

In [ ]:
# 分割训练集和测试集
X_train = X[:80]
y_train = y[:80]
X_test = X[80:]
y_test = y[80:]

train_errors = []
test_errors = []
degrees = range(1, 11)

for degree in degrees:
    # 创建多项式特征
    X_train_poly = create_polynomial_features(X_train, degree)
    X_test_poly = create_polynomial_features(X_test, degree)
    
    # 训练模型
    theta = linear_regression(X_train_poly, y_train)
    
    # 计算训练集和测试集的MSE
    y_train_pred = predict(X_train_poly, theta)
    y_test_pred = predict(X_test_poly, theta)
    
    train_errors.append(compute_mse(y_train, y_train_pred))
    test_errors.append(compute_mse(y_test, y_test_pred))

print("不同阶数多项式回归的表现:")
for degree, train_err, test_err in zip(degrees, train_errors, test_errors):
    print(f"{degree}阶多项式 - 训练MSE: {train_err:.4f}, 测试MSE: {test_err:.4f}")

## 8. 可视化过拟合现象

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(degrees, train_errors, 'b-o', label='训练误差')
plt.plot(degrees, test_errors, 'r-o', label='测试误差')
plt.xlabel('多项式阶数')
plt.ylabel('均方误差 (MSE)')
plt.title('多项式回归的过拟合分析')
plt.legend()
plt.grid(True)
plt.show()

## 9. 选择最优多项式阶数

In [ ]:
# 找到测试误差最小的多项式阶数
optimal_degree = degrees[np.argmin(test_errors)]
min_test_error = min(test_errors)

print(f"最优多项式阶数: {optimal_degree}")
print(f"对应的测试误差: {min_test_error:.4f}")

# 训练最优模型
X_poly_optimal = create_polynomial_features(X, optimal_degree)
theta_optimal = linear_regression(X_poly_optimal, y)

# 可视化最优模型
X_new = np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = create_polynomial_features(X_new, optimal_degree)
y_pred_optimal = predict(X_new_poly, theta_optimal)

plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, label='训练数据')
plt.plot(X_new, y_pred_optimal, 'r-', linewidth=2, label=f'最优模型 ({optimal_degree}阶多项式)')
plt.xlabel('特征 X')
plt.ylabel('目标值 y')
plt.title(f'最优多项式回归模型 (测试MSE: {min_test_error:.4f})')
plt.legend()
plt.grid(True)
plt.show()

## 10. 多项式回归总结

In [ ]:
print("多项式回归的优缺点:")
print("优点:")
print("1. 可以捕捉非线性关系")
print("2. 实现简单，基于线性回归")
print("3. 灵活性高，可以通过调整阶数适应不同复杂度的数据")

print("\n缺点:")
print("1. 容易过拟合，尤其是高阶多项式")
print("2. 计算复杂度随阶数增加而增加")
print("3. 外推能力有限，在训练数据范围外可能表现很差")

print("\n避免过拟合的方法:")
print("- 选择合适的多项式阶数")
print("- 使用交叉验证评估模型")
print("- 考虑使用正则化方法")
print("- 确保有足够的训练数据")